## Data cleasning and preprocessing of the option pricing data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import yfinance as yf


/Users/ningsam/anaconda3/envs/YichenWang/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/3w/bkk9z93s1294lzphg00_yxnw0000gn/T/ipykernel_44034/2251265277.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Read in the data from Jan 2023 to June 2023
df_2023_h1 = pd.DataFrame()
for i in [202301, 202302, 202303, 202304,  202305]:
    df_2023_h1 = pd.concat([df_2023_h1, pd.read_table(f'data/spy_eod_{i}.txt', sep=',')], ignore_index=True)
df_2023_h1.columns = df_2023_h1.columns.str.strip()

/var/folders/3w/bkk9z93s1294lzphg00_yxnw0000gn/T/ipykernel_44034/79976602.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023_h1 = pd.concat([df_2023_h1, pd.read_table(f'data/spy_eod_{i}.txt', sep=',')], ignore_index=True)


In [4]:
# also drop expiration date later than 2024
df_2023_h1 = df_2023_h1[df_2023_h1['[EXPIRE_DATE]'] <= ' 2023-12-31']
df_2023_h1 = df_2023_h1.reset_index()

In [5]:
# get target data from yfinance
target = pd.DataFrame(yf.download(['SPY'], start="2023-01-01", end="2023-12-31")['Adj Close'])
target

/Users/ningsam/anaconda3/envs/YichenWang/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


,Adj Close
Date,
2023-01-03,375.118713
2023-01-04,378.014740
2023-01-05,373.700256
2023-01-06,382.269989
2023-01-09,382.053284
...,...
2023-12-22,473.649994
2023-12-26,475.649994
2023-12-27,476.510010


In [6]:
df_2023_h1['[EXPIRE_DATE]'] = df_2023_h1['[EXPIRE_DATE]'].str.strip().astype('datetime64[ns]')
df_2023_h1['[EXPIRE_DATE]'].unique()

<DatetimeArray>
['2023-01-03 00:00:00', '2023-01-04 00:00:00', '2023-01-05 00:00:00',
 '2023-01-06 00:00:00', '2023-01-09 00:00:00', '2023-01-10 00:00:00',
 '2023-01-11 00:00:00', '2023-01-12 00:00:00', '2023-01-13 00:00:00',
 '2023-01-20 00:00:00',
 ...
 '2023-06-01 00:00:00', '2023-06-05 00:00:00', '2023-06-06 00:00:00',
 '2023-06-07 00:00:00', '2023-06-08 00:00:00', '2023-07-07 00:00:00',
 '2023-06-12 00:00:00', '2023-06-13 00:00:00', '2023-06-14 00:00:00',
 '2023-11-17 00:00:00']
Length: 125, dtype: datetime64[ns]

In [7]:
target['[EXPIRE_DATE]'] = target.index
target['[EXPIRE_DATE]'].astype('datetime64[ns]')

Date
2023-01-03   2023-01-03
2023-01-04   2023-01-04
2023-01-05   2023-01-05
2023-01-06   2023-01-06
2023-01-09   2023-01-09
                ...    
2023-12-22   2023-12-22
2023-12-26   2023-12-26
2023-12-27   2023-12-27
2023-12-28   2023-12-28
2023-12-29   2023-12-29
Name: [EXPIRE_DATE], Length: 250, dtype: datetime64[ns]

In [8]:
df_2023_h1 = pd.merge(df_2023_h1, target, on = '[EXPIRE_DATE]')

In [10]:
df_2023_h1.columns

Index(['index', '[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
       '[QUOTE_TIME_HOURS]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]',
       '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]',
       '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_SIZE]',
       '[P_LAST]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]', 'Adj Close'],
      dtype='object')

In [22]:
df_2023_h1['maturity_time'] = df_2023_h1['[EXPIRE_DATE]'] - df_2023_h1['[QUOTE_DATE]'].astype('datetime64[ns]')
# Convert the Timedelta to total seconds and then divide by the number of seconds in a day to get days as a float
df_2023_h1['maturity_time'] = df_2023_h1['maturity_time'].dt.total_seconds() / (60 * 60 * 24 * 365)


In [29]:
import math
df_2023_h1['discounted_price'] = df_2023_h1['Adj Close']
for i in range(len(df_2023_h1)):
    df_2023_h1['discounted_price'] = (df_2023_h1['[STRIKE]']-df_2023_h1['Adj Close'])*math.exp(df_2023_h1['maturity_time'][i]*(-0.04))

In [30]:
df_2023_h1

,index,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],...,[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT],Adj Close,maturity_time,discount_rate,discounted_price
0,0,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.96551,...,-0.00483,-0.00015,1.210050,0.000000,70.8,0.186,375.118713,0.000000,1.0,-63.623257
1,1,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.96015,...,-0.00487,0.00000,0.996160,0.000000,60.8,0.160,375.118713,0.000000,1.0,-53.852908
2,2,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.95788,...,-0.00538,-0.00007,0.911990,,56.8,0.149,375.118713,0.000000,1.0,-49.944768
3,3,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.96337,...,-0.00500,-0.00048,0.890650,0.000000,55.8,0.147,375.118713,0.000000,1.0,-48.967734
4,4,1672779600,2023-01-03 16:00,2023-01-03,16.0,380.82,2023-01-03,1672779600,0.00,0.95600,...,-0.00533,-0.00044,0.870040,0.000000,54.8,0.144,375.118713,0.000000,1.0,-47.990699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331604,403301,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-12-29,1703883600,212.04,0.02990,...,-0.00398,-0.20673,0.204150,,92.2,0.221,475.309998,0.580822,1.0,33.893344
331605,403302,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-12-29,1703883600,212.04,0.02566,...,-0.00410,-0.20806,0.212500,,97.2,0.233,475.309998,0.580822,1.0,38.778519
331606,403303,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-12-29,1703883600,212.04,0.02121,...,-0.00467,-0.20572,0.219890,0.000000,102.2,0.245,475.309998,0.580822,1.0,43.663693
331607,403304,1685563200,2023-05-31 16:00,2023-05-31,16.0,417.80,2023-12-29,1703883600,212.04,0.01735,...,-0.00476,-0.20165,0.227210,0.000000,107.2,0.257,475.309998,0.580822,1.0,48.548868


In [5]:
# ignoring underlying interest rate, lets just use direct profit comparison to try first
# we can use target price (real adj close price) - strike price as our to be predicted quote price. 
# still need to fit target into our original dataframe tho. 
# df_2023_h1.join(target, on= 'Date') # Date = Expire Date here. 

In [6]:
"""# Now do some feature engineering. First deal with moneyness. Moneyness, # probably change it to boolean? 
# by definition, refers to the strike price of an option relative to the current trading price of the underlying asset.
df_2023_h1['Moneyness'] = 'ATM'
df_2023_h1.loc[df_2023_h1['[STRIKE]'] < df_2023_h1['[UNDERLYING_LAST]'], 'Moneyness'] = 'ITM'
df_2023_h1.loc[df_2023_h1['[STRIKE]'] > df_2023_h1['[UNDERLYING_LAST]'], 'Moneyness'] = 'OTM'"""

"# Now do some feature engineering. First deal with moneyness. Moneyness, # probably change it to boolean? \n# by definition, refers to the strike price of an option relative to the current trading price of the underlying asset.\ndf_2023_h1['Moneyness'] = 'ATM'\ndf_2023_h1.loc[df_2023_h1['[STRIKE]'] < df_2023_h1['[UNDERLYING_LAST]'], 'Moneyness'] = 'ITM'\ndf_2023_h1.loc[df_2023_h1['[STRIKE]'] > df_2023_h1['[UNDERLYING_LAST]'], 'Moneyness'] = 'OTM'"

In [31]:
df_2023_h1.columns  # someone pls check this and fill it into the standard scalar

Index(['index', '[QUOTE_UNIXTIME]', '[QUOTE_READTIME]', '[QUOTE_DATE]',
       '[QUOTE_TIME_HOURS]', '[UNDERLYING_LAST]', '[EXPIRE_DATE]',
       '[EXPIRE_UNIX]', '[DTE]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]', '[C_LAST]', '[C_SIZE]',
       '[C_BID]', '[C_ASK]', '[STRIKE]', '[P_BID]', '[P_ASK]', '[P_SIZE]',
       '[P_LAST]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[STRIKE_DISTANCE]',
       '[STRIKE_DISTANCE_PCT]', 'Adj Close', 'maturity_time', 'discount_rate',
       'discounted_price'],
      dtype='object')

In [32]:
df_2023_h1 = df_2023_h1[['[QUOTE_UNIXTIME]', '[EXPIRE_UNIX]', '[STRIKE]', '[UNDERLYING_LAST]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]','[C_BID]', '[C_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[P_BID]', '[P_ASK]', 'discounted_price']]
for column in df_2023_h1.columns:
    print(type(df_2023_h1[column][0]))

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [33]:
df_2023_h1 = df_2023_h1.replace(r'^\s*$', 0, regex=True)

In [34]:
for column in ['[C_IV]', '[C_VOLUME]', '[P_IV]', '[P_VOLUME]']:
    df_2023_h1[column] = df_2023_h1[column].str.strip().astype('float64')
    print(type(df_2023_h1[column][0]))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [35]:
df_2023_h1 = df_2023_h1.fillna(0)

In [36]:
# Basic normalization and standardization
# run block of code and catch warnings
import warnings
from sklearn.preprocessing import StandardScaler
with warnings.catch_warnings():
	# ignore all caught warnings
	warnings.filterwarnings("ignore")
	# execute code that will generate warnings
	numeric_cols = ['[QUOTE_UNIXTIME]', '[EXPIRE_UNIX]', '[STRIKE]', '[UNDERLYING_LAST]', '[C_DELTA]', '[C_GAMMA]', '[C_VEGA]',
       '[C_THETA]', '[C_RHO]', '[C_IV]', '[C_VOLUME]','[C_BID]', '[C_ASK]', '[P_DELTA]', '[P_GAMMA]', '[P_VEGA]', '[P_THETA]',
       '[P_RHO]', '[P_IV]', '[P_VOLUME]', '[P_BID]', '[P_ASK]']  # not sure about all this, we ball
	scaler = StandardScaler()
	df_2023_h1[numeric_cols] = scaler.fit_transform(df_2023_h1[numeric_cols])

In [37]:
df_2023_h1

,[QUOTE_UNIXTIME],[EXPIRE_UNIX],[STRIKE],[UNDERLYING_LAST],[C_DELTA],[C_GAMMA],[C_VEGA],[C_THETA],[C_RHO],[C_IV],...,[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[P_BID],[P_ASK],discounted_price
0,-1.69160,-1.531564,-1.054517,-2.406592,1.054125,-0.052714,0.000146,-0.391189,-0.307878,11.711458,...,1.120791,-0.013192,0.001135,0.086874,0.592295,3.380989,-0.100406,-0.563368,-0.566211,-63.623257
1,-1.69160,-1.531564,-0.936052,-2.406592,1.041020,0.036229,0.000190,-0.390353,-0.308470,10.342762,...,1.120350,-0.013031,0.001152,0.086843,0.592878,2.599981,-0.100406,-0.563368,-0.566211,-53.852908
2,-1.69160,-1.531564,-0.888665,-2.406592,1.035470,0.083539,0.000199,-0.393217,-0.308536,9.795964,...,1.119196,-0.013111,0.001152,0.086442,0.592606,2.292638,-0.100406,-0.563368,-0.566211,-49.944768
3,-1.69160,-1.531564,-0.876819,-2.406592,1.048893,0.103725,0.000140,0.031159,-0.308217,9.553426,...,1.120644,-0.013015,0.001157,0.086741,0.591011,2.214716,-0.100406,-0.563368,-0.566211,-48.967734
4,-1.69160,-1.531564,-0.864972,-2.406592,1.030873,0.108141,0.000225,-0.438792,-0.308574,9.529689,...,1.119712,-0.013079,0.001120,0.086482,0.591167,2.139460,-0.100406,-0.563368,-0.566211,-47.990699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331604,1.78216,2.332822,1.314799,1.410238,-1.233498,-0.296836,0.011186,0.782436,-0.241783,-0.825637,...,-1.252159,-0.005506,0.009906,0.087542,-0.211005,-0.292004,-0.100406,1.159106,1.213589,33.893344
331605,1.78216,2.332822,1.374032,1.410238,-1.243865,-0.311975,0.009622,0.786493,-0.252636,-0.825637,...,-1.248011,-0.006006,0.009987,0.087448,-0.216177,-0.261514,-0.100406,1.254270,1.308039,38.778519
331606,1.78216,2.332822,1.433265,1.410238,-1.254745,-0.327114,0.008129,0.801048,-0.262718,-0.825637,...,-1.248060,-0.006521,0.009984,0.087000,-0.207077,-0.234530,-0.100406,1.349435,1.402301,43.663693
331607,1.78216,2.332822,1.492498,1.410238,-1.264183,-0.337838,0.006965,0.808326,-0.269625,-0.825637,...,-1.246833,-0.006876,0.009942,0.086929,-0.191251,-0.207801,-0.100406,1.444218,1.496751,48.548868


In [14]:
# Extract features from datetime and encode categorical features
# Do we still use datetime ?? or just use unix 
"""df_2023_h1['[QUOTE_MONTH]'] = df_2023_h1['[QUOTE_DATE]'].dt.month
df_2023_h1['[EXPIRE_WEEKDAY]'] = df_2023_h1['[EXPIRE_DATE]'].dt.weekday
df_2023_h1=pd.get_dummies(df_2023_h1, columns = ['Moneyness'])"""

"df_2023_h1['[QUOTE_MONTH]'] = df_2023_h1['[QUOTE_DATE]'].dt.month\ndf_2023_h1['[EXPIRE_WEEKDAY]'] = df_2023_h1['[EXPIRE_DATE]'].dt.weekday\ndf_2023_h1=pd.get_dummies(df_2023_h1, columns = ['Moneyness'])"

In [40]:
# output the df_2023_h1 to a csv file, output to the desktop
df_2023_h1.to_csv(r'/Users/ningsam/Desktop/Winter_2024/PIC16B/df_2023_h1.csv', index = False, header=True)